In [1]:
import numpy as np
import networkx as nx

In [106]:
def get_motif_abs(graph):
    """Return a list of motif absolute occurences
    
    graph: the graph
    """
    # get all triads
    triads = nx.triadic_census(graph)
    
    # remove first three triads 
    triads_to_remove = ('003', '012', '102')
    for k in triads_to_remove:
        triads.pop(k, None)
    
    return list(triads.values())

def get_motif_sig_profile(graph, rand_num):
    """Returns a the graph motif significance profile
    
    graph: the graph
    rand_num: the number of random graphs to compute 
    """    
    triads = get_motif_abs(graph)
    
    # compute rand_num significance profiles of random graphs
    rand_triads = [] 
    
    for i in range(rand_num):
        # generate random configuration model
        din = list(d for n, d in graph.in_degree())
        dout = list(d for n, d in graph.out_degree())
        rand_graph = nx.directed_configuration_model(din, dout, create_using=nx.DiGraph())

        rand_triads.append(get_motif_abs(rand_graph))
    
    rand_triads = np.array(rand_triads)
    
    rand_mean = rand_triads.mean(axis=0)
    rand_std = rand_triads.std(axis=0)
    
    # divide and check for zeros 
    a = (triads - rand_mean)
    b = rand_std
    z_scores = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    
    # normalize Z scores 
    a = z_scores
    b = np.sqrt(np.sum(z_scores**2))
    z_scores = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
        
    return z_scores.tolist()

In [107]:
# create a test directed graph 
G = nx.fast_gnp_random_graph(50, 0.2, directed=True)

In [108]:
get_motif_sig_profile(G, 100)

[0.3306748663801864,
 0.2955430949469171,
 0.2616403117313315,
 0.24255660015892375,
 0.2184249091947977,
 0.36391626355388623,
 0.2848957421791377,
 0.05586295465853454,
 0.32704006921276635,
 0.2922784384518591,
 0.44650680607205867,
 0.094721399495001,
 0.10423986508935924]